![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column                   | Description                                                                      |
|------------------------- |--------------------------------------------------------------------------------- |
| `student_id`             | A unique ID for each student.                                                    |
| `city`                   | A code for the city the student lives in.                                        |
| `city_development_index` | A scaled development index for the city.                                         |
| `gender`                 | The student's gender.                                                            |
| `relevant_experience`    | An indicator of the student's work relevant experience.                          |
| `enrolled_university`    | The type of university course enrolled in (if any).                              |
| `education_level`        | The student's education level.                                                   |
| `major_discipline`       | The educational discipline of the student.                                       |
| `experience`             | The student's total work experience (in years).                                  |
| `company_size`           | The number of employees at the student's current employer.                       |
| `company_type`           | The type of company employing the student.                                       |
| `last_new_job`           | The number of years between the student's current and previous jobs.             |
| `training_hours`         | The number of hours of training completed.                                       |
| `job_change`             | An indicator of whether the student is looking for a new job (`1`) or not (`0`). |

In [328]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
ds_jobs = pd.read_csv("customer_train.csv")

# View the dataset
ds_jobs.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [329]:
# Create a copy of ds_jobs for transforming
ds_jobs_transformed = ds_jobs.copy()

ds_jobs_transformed.dtypes

student_id                  int64
city                       object
city_development_index    float64
gender                     object
relevant_experience        object
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours              int64
job_change                float64
dtype: object

### setting integer and float types

In [330]:
columns = ds_jobs_transformed.columns.tolist()

# setting int64 to int32 and float64 to float 16
for col in columns:
    if ds_jobs_transformed[col].dtype == 'int64':
        ds_jobs_transformed[col] = ds_jobs_transformed[col].astype('int32')
    elif ds_jobs_transformed[col].dtype == 'float64':
        ds_jobs_transformed[col] = ds_jobs_transformed[col].astype('float16')

ds_jobs_transformed.dtypes

student_id                  int32
city                       object
city_development_index    float16
gender                     object
relevant_experience        object
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours              int32
job_change                float16
dtype: object

### setting columns with only two unique factors to Boolean

In [331]:
two_factor_columns = []
for col in columns:
    if ds_jobs_transformed[col].nunique() == 2:
        two_factor_columns.append(col)

for col in two_factor_columns:
    ds_jobs_transformed[col] = ds_jobs_transformed[col].astype('bool')

ds_jobs_transformed.dtypes

student_id                  int32
city                       object
city_development_index    float16
gender                     object
relevant_experience          bool
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours              int32
job_change                   bool
dtype: object

### setting nominal categorical columns to 'category' dtype

In [332]:
threshold = len(ds_jobs_transformed)*0.01      # threshold is 1% of the length of the DataFrame
cat_columns = []

# removing boolean and numeric columns from columns so that they don't get set to 'category'
for col in columns:
    if ds_jobs_transformed[col].dtype == 'bool':
        columns.remove(col)
        
# if the unique values in a column is less than, or equal to threshold, the column is nominal categorical
for col in columns:
    if ds_jobs_transformed[col].nunique() <= threshold:
        cat_columns.append(col)

# setting them to dtype 'category'
for col in cat_columns:
    ds_jobs_transformed[col] = ds_jobs_transformed[col].astype('category')

# city_development_index has to remain float16
ds_jobs_transformed['city_development_index'] = ds_jobs_transformed['city_development_index'].astype('float16')

ds_jobs_transformed.dtypes

student_id                   int32
city                      category
city_development_index     float16
gender                    category
relevant_experience           bool
enrolled_university       category
education_level           category
major_discipline          category
experience                category
company_size              category
company_type              category
last_new_job              category
training_hours               int32
job_change                    bool
dtype: object

### setting ordered categories to be ordinal

In [333]:
ord_columns = ['education_level', 'experience', 'company_size', 'last_new_job', 'enrolled_university']

# # setting 'education'
edu_order = ['Primary School', 'High School', 'Graduate', 'Masters', 'Phd']
ds_jobs_transformed['education_level'] = ds_jobs_transformed['education_level'].cat.set_categories(edu_order, ordered=True)

# getting the value counts
ds_jobs_transformed['education_level'].value_counts(sort=False)

Primary School      308
High School        2017
Graduate          11598
Masters            4361
Phd                 414
Name: education_level, dtype: int64

In [334]:
# setting enrolled_university

# setting order
uni_order = ['no_enrollment', 'Part time course', 'Full time course']

# ordering the column
ds_jobs_transformed['enrolled_university'] = ds_jobs_transformed['enrolled_university'].cat.set_categories(uni_order, ordered=True)

# getting the value_count
ds_jobs_transformed['enrolled_university'].value_counts(sort=False)

no_enrollment       13817
Part time course     1198
Full time course     3757
Name: enrolled_university, dtype: int64

In [335]:
exp = ['<1', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15',
       '16', '17', '18', '19', '20', '>20']

ds_jobs_transformed['experience'] = ds_jobs_transformed['experience'].astype(
    pd.CategoricalDtype(categories=exp, ordered=True)
)

company_size_order = [
    '<10', '10-49', '50-99', '100-499', '500-999', '1000-4999', '5000-9999', '10000+'
]
ds_jobs_transformed['company_size'] = ds_jobs_transformed['company_size'].astype(
    pd.CategoricalDtype(categories=company_size_order, ordered=True)
)

print(ds_jobs_transformed['experience'].dtype)
print(ds_jobs_transformed['company_size'].dtype)


category
category


In [336]:
# setting last_new_job

# renamin 'never' to '0'
ds_jobs_transformed['last_new_job'] = ds_jobs_transformed['last_new_job'].replace('never', '0')

# defining order
job_order = ['0', '1', '2', '3','4', '>4']

# ordering the column
ds_jobs_transformed['last_new_job'] = ds_jobs_transformed['last_new_job'].cat.set_categories(job_order, ordered=True)

# the column
ds_jobs_transformed['last_new_job'].value_counts(sort=False)

0     2452
1     8040
2     2900
3     1024
4     1029
>4    3290
Name: last_new_job, dtype: int64

### Filtering the DataFrame

In [337]:
ds_jobs_transformed = ds_jobs_transformed[
    (ds_jobs_transformed['experience'] >= '10') &
    (ds_jobs_transformed['company_size'] >= '1000-4999')
]


In [338]:
print(ds_jobs_transformed['experience'].dtype)
print(ds_jobs_transformed['company_size'].dtype)


category
category


In [341]:
ds_jobs_transformed.shape

(2201, 14)

<div style='background-color:#fff; color:#000; font-family: "Georgia"; font-size:16px;'>The preprocessing has reduced the memory size of the DataFrame by keeping only the relevant rows that the recruiter base is interested in hiring, making the job of company much easier. This is the task of data scientist: to reduce the workload of company and streamline processes.</div>